## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [8]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
     users.append(agg['user__username'])   
users.remove("jmperez")

users

['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

In [16]:


base_query = {
    "user__username__in": users,
    "article__batch__name": "training",
}
ArticleLabel.objects.filter(**base_query).count()

78

¿Cuántos etiquetamos todes?

In [17]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Escándalo en España: fallan los tests de coronavirus fabricados en China,El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,"Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina""",Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad”,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras,Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero",Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,Intentó violar a su novia y ella lo asesinó a escobazos
name,,,,,,,,,,,,,
asilva,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
dvalado,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
fdesanctis,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
gclerici,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
gdamill,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
lprats,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [19]:
article_labels = ArticleLabel.objects.filter(
    **base_query,
)

skipped_articles = [
    #Article.objects.get(title__contains="Thelma Fardin contó cómo sigue la causa contra Juan Darthés").id,
    #Article.objects.get(title__contains="Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria").id,
]

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    if article_label.article.id in skipped_articles:
        continue
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,341401,341402,341403,341404,341405,341406,341407,341408,341409,341410,341411,341412,341413,341414,341415,341416,341417,341418,341419,341420,...,370532,370533,370534,370535,370536,370537,370538,370539,370540,370541,370542,370543,370544,370545,370546,370547,370548,370549,370550,370551
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
asilva,True,True,False,False,True,True,False,True,False,False,False,False,False,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
dvalado,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,True,False,True
fdesanctis,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,True,False
gclerici,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,True,False,False,True,...,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
gdamill,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
lprats,True,True,False,False,True,True,False,True,False,False,False,False,False,False,True,False,False,False,False,True,...,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False


In [20]:
from nltk import agreement

def kappa_of(df, subset=None):
    if subset:
        coders = df.loc[subset]
    else:
        coders = df
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(df_comments, subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(df_comments, subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para asilva dvalado fdesanctis gclerici gdamill lprats = 0.619 (#instancias = 563)


Pares


Fleiss K para asilva dvalado                           = 0.612 (#instancias = 611)
Fleiss K para asilva fdesanctis                        = 0.605 (#instancias = 611)
Fleiss K para asilva gclerici                          = 0.542 (#instancias = 563)
Fleiss K para asilva gdamill                           = 0.674 (#instancias = 611)
Fleiss K para asilva lprats                            = 0.574 (#instancias = 611)
Fleiss K para dvalado fdesanctis                       = 0.638 (#instancias = 611)
Fleiss K para dvalado gclerici                         = 0.596 (#instancias = 563)
Fleiss K para dvalado gdamill                          = 0.717 (#instancias = 611)
Fleiss K para dvalado lprats                           = 0.646 (#instancias = 611)
Fleiss K para fdesanctis gclerici                      = 0.556 (#instancias = 563)
Fleiss K para fdesanctis gdamill                       = 0.649 (#ins

Podemos calcularlo con krippendorff ya que algunos saltearon

In [21]:
import krippendorff

alpha = krippendorff.alpha(df_comments.values.astype('float'))

print(f"Alpha para {' - '.join(users)}: {alpha:.3f}")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        
        alpha = krippendorff.alpha(df_comments.loc[subset].values.astype('float'))
        print(f"Alpha para {name:<40} = {alpha:.3f}")



Alpha para asilva - dvalado - fdesanctis - gclerici - gdamill - lprats: 0.616
Alpha para asilva dvalado                           = 0.608
Alpha para asilva fdesanctis                        = 0.602
Alpha para asilva gclerici                          = 0.537
Alpha para asilva gdamill                           = 0.672
Alpha para asilva lprats                            = 0.572
Alpha para dvalado fdesanctis                       = 0.638
Alpha para dvalado gclerici                         = 0.597
Alpha para dvalado gdamill                          = 0.718
Alpha para dvalado lprats                           = 0.646
Alpha para fdesanctis gclerici                      = 0.556
Alpha para fdesanctis gdamill                       = 0.649
Alpha para fdesanctis lprats                        = 0.626
Alpha para gclerici gdamill                         = 0.579
Alpha para gclerici lprats                          = 0.569
Alpha para gdamill lprats                           = 0.657


In [22]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
asilva        120.0
dvalado       176.0
fdesanctis    165.0
gclerici      169.0
gdamill       161.0
lprats        158.0
dtype: float64

## Agreements sobre cada categoría

In [23]:
idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)


In [25]:

article_labels = ArticleLabel.objects.filter(
    **base_query,
)


for article_label in article_labels:
    username = article_label.user.username
    
    if article_label.article.id in skipped_articles:
        continue
        
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

#df_labels

In [26]:
print("##", ' - '.join(users), '\n')

#users = ["mkondra", "jmperez"]

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category].loc[users])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## asilva - dvalado - fdesanctis - gclerici - gdamill - lprats 

| MUJER          |  0.519
| LGBTI          |  0.749
| RACISMO        |  0.646
| POBREZA        |  0.645
| POLITICA       |  0.456
| DISCAPACIDAD   |  0.653
| ASPECTO        |  0.343
| CRIMINAL       |  0.787
No hay elementos para calcular Kappa de OTROS - salteando
Instancias: 563


## dvalado - gdamill - gclerici - fdesanctis 

| Anotador       | Kappa |
|----------------|-------|
| MUJER          |  0.583|
| LGBTI          |  0.740|
| RACISMO        |  0.644|
| POBREZA        |  0.613|
| POLITICA       |  0.482|
| DISCAPACIDAD   |  0.638|
| ASPECTO        |  0.514|
| CRIMINAL       |  0.740|

## dvalado - gdamill - gclerici 
| Anotador       |  Kappa| 
|----------------|-------|
| MUJER          |  0.575| 
| LGBTI          |  0.759| 
| RACISMO        |  0.737| 
| POBREZA        |  0.607| 
| POLITICA       |  0.564| 
| DISCAPACIDAD   |  0.570| 
| ASPECTO        |  0.610| 
| CRIMINAL       |  0.797| 
No hay elementos para calcular Kappa de OTROS - salteando

## Alpha Krippendorff

In [ ]:
import krippendorff

users = list(df_labels.loc["MUJER"].index)

user_combinations = [users]
user_combinations += [[u1, u2] for u1, u2 in set((u1, u2) for u1 in users for u2 in users if u1 != u2 and u1 < u2)]

for user_list in user_combinations:
    print("="*80 + '\n' * 2)
    print(" - ".join(user_list), "\n")
    print("| Etiquetador    |        |")
    print("|----------------|--------|")
    for category in CommentLabel.type_mapping:
        mat = df_labels.loc[category].loc[user_list].values
        mat = mat.astype(float)
        alpha = krippendorff.alpha(mat)
        print(f"| {category:<12}   |  {alpha:.3f} |")



asilva - dvalado - fdesanctis - gclerici - gdamill - lprats 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.507 |
| LGBTI          |  0.736 |
| RACISMO        |  0.599 |
| POBREZA        |  0.720 |
| POLITICA       |  0.409 |
| DISCAPACIDAD   |  0.635 |
| ASPECTO        |  0.500 |
| CRIMINAL       |  0.754 |
| OTROS          |  -0.000 |

dvalado - gdamill - gclerici - fdesanctis - asilva 

| Clase          | Alpha  |
|----------------|--------|
| MUJER          |  0.566 |
| LGBTI          |  0.728 |
| RACISMO        |  0.605 |
| POBREZA        |  0.619 |
| POLITICA       |  0.469 |
| DISCAPACIDAD   |  0.673 |
| ASPECTO        |  0.364 |
| CRIMINAL       |  0.759 |
| OTROS          |  nan   |




asilva - fdesanctis - gdamill - lprats 

| Clase          | Acuerdo | 
|----------------|---------|
| MUJER          |  0.501  |
| LGBTI          |  0.772  |
| RACISMO        |  0.635  |
| POBREZA        |  0.499  |
| POLITICA       |  0.436  |
| DISCAPACIDAD   |  0.462  |
| ASPECTO        |  0.517  |
| CRIMINAL       |  0.931  |
| OTROS          |  -0.002 |


Apreciaciones generales:

- CRIMINAL: altísimo acuerdo
- MUJER: acuerdo razonable, pero hay que mejorar
- LGBTI y RACISMO: acuerdo relativamente alto
- POBREZA: ver qué pasa acá (ver Gabriel y Alan que etiquetaron lo mismo)
- POLITICA: esperable bajo acuerdo
- ASPECTO: ver acá qué pasó
- DISCAPACIDAD: ver también
- OTROS: ¿qué marcaron?




## MUJER


In [28]:
from collections import defaultdict

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

            
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*45 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in sorted(comments, key=lambda c: sum(df[c.id])):
            print("")
            print("  - ",comment.text, end="")
            if show:
                yes = [annotator for annotator, v in df[comment.id].items() if v] 
                no = [annotator for annotator, v in df[comment.id].items() if not v] 
                print(f"\n     - SI: {yes}  NO: {no}")
            else:
                print("")

In [29]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    non_hateful_comments = category_df[category_df.columns[(~category_df.astype(bool)).all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    
    disagree_comments = category_df[category_df.columns[no_agree]]
    # Estos son muchos! Filtrar sólo los de aquellos artículos
    # donde haya habido algún acuerdo
    
    allowed_articles = set(
        c.article_id for c in Comment.objects.filter(id__in=agree_comments.columns)
    )
    
    allowed_comments = Comment.objects.filter(
        id__in=non_hateful_comments.columns,
        article_id__in=allowed_articles,
    )
    
    #print(len(allowed_comments))
    comment_ids = set(c.id for c in allowed_comments)
    
    non_hateful_comments = non_hateful_comments[[
        c for c in non_hateful_comments.columns if c in comment_ids
    ]]
    return agree_comments, disagree_comments, non_hateful_comments


agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "MUJER", users
)

In [30]:
show_from_df(agree_comments)


"Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero 


  -  @usuario Lindo gatito

  -  @usuario Esa mentalidad de la mujer latina de mostrar las tetas como ofreciendo la mercadería al mejor postor

  -  @usuario Si tenes que mostrar las tetas para conducir un programa es porque no tenes ninguna aptitud para conducir. Eso pasa cuando queres que te miren y no te escuchen porque sos pésima. No veo a periodistas o conductoras de trayectoria necesitar salir en pelotas.

  -  @usuario Sos un gato más! En vez de sobresalir por ser una brillante periodista, te metes en boca de todos por la tetas que mostraste ( que dicho sea de paso son artificiales). Una de las tantas más que tira para abajo empobreciendo la tv

  -  @usuario Evolucionen te tira está que quiere tapar su falta de preparación mostrando el cuerpo cuál vedetonga? 🤣

  -  @usuario La manera de justificar su promiscuidad, es decir que la gente no evoluciona ? Jajajja. 

Busquemos dónde hay discrepancias

In [32]:
show_from_df(disagree_comments, show=True)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario La rabina ,si es que eso existe,deberá estudiar en el Toráh la opinión que tiene yaveh acerca de la sodomía.....hagan lo que se les cante pero no sé metan con creencias milenarias.....dios no ama a los promiscuos
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favoreció al país para q los maracas y lesbianas se casen, y eso la hace una gran persona, jajajaja mientras t chorea todo!!!
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Que la chupe
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Para

In [34]:
%%capture
show_from_df(non_hateful_comments)

## LGBTI


In [35]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "LGBTI", users
)

In [36]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario asco

  -  @usuario 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river

  -  @usuario 🤢

  -  @usuario Aféitense, chicas! 💋

  -  @usuario ups el barba era trolis ... ups y parecía gente de bien

  -  @usuario Asco

  -  @usuario Asco

  -  @usuario La pucha, parece que son más los anormales. 
Hacia donde va la humanidad?
Que dos anormales se juntan, es noticia en un diario.

  -  @usuario Y a quién joraca le importa q estos 2 putitos se casen, encima se sacan selfies, q manga d trolos asquerosos culos rotos!! Como les gusta ventilar la mariconeada, se creen q uno está pendiente d sus vidas?!!


In [38]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Y la culp es de españa ?  O los putos chinos ?
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Que cosa peluda 🦍😎
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Completo
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario 😝q carajo nos importa
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario En ningun parrafo de los 70 libros que fueron escritos por inspiracion divina y en estos libros sagrados solamente figuran UNGIDOS los varones para el sacerdocio...!?
     - SI: ['dvalado', 'lprats']  NO: ['asilva', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Y que carajo me importa lo que hagan con sus C.
   

In [39]:
%%capture
show_from_df(non_hateful_comments)

## RACISMO

In [40]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "RACISMO", users
)

In [41]:
show_from_df(agree_comments)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Y la culp es de españa ?  O los putos chinos ?

  -  @usuario Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto

  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN

  -  @usuario Estos chinos son una mierda

Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos 


  -  @usuario Negra cloaquera

  -  @usuario tremenda negra boluda

  -  @usuario Negra y abortera


In [43]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Chinverguenzas
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario @usuario Los cagaron otra vez
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Estos son socios del que hizo el virus!!!!
     - SI: ['dvalado', 'gclerici']  NO: ['asilva', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario China va a tener que pagarle al mindo todo el daño causado
     - SI: ['dvalado', 'gclerici']  NO: ['asilva', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario No podía fallar!  Jajajajaja Perdón que me ria pero jamás les hubiera comprado a ellos!
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Los chinos van por todos nosotros.Necesitan espacio !!!
     - SI: ['dvalado', 'gclerici', 'gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

  -  @usuario @usuario Per

In [44]:
%%capture
show_from_df(non_hateful_comments)

## POBREZA

In [45]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POBREZA", users
)

In [46]:
show_from_df(agree_comments)


Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras 


  -  @usuario BALA

  -  @usuario VAGOS !! Usurpadores

  -  @usuario Bala y al ✨río✨

  -  @usuario Es el momento de tirar una bomba y que exploten todos


In [47]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 


  -  @usuario Pero GGG, las imágenes del conurbano, especialmente en La Matanza son del final absoluto de la cuarentena, todo el mundo en la calle, ferias de manteros, parece el inicio de temporada de saldos! ESO les tiene q preocupar!  Andar en bicicleta o correr de noche es un poco de salud!
     - SI: ['dvalado']  NO: ['asilva', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Y si, al enano de Mierda le molestaba que los tipos salgan a correr, mientras los indios del congourbano están encerrados.
     - SI: ['asilva', 'dvalado', 'gdamill', 'lprats']  NO: ['fdesanctis', 'gclerici']

Protestas

In [48]:
%%capture
show_from_df(non_hateful_comments)

## POLITICA

In [49]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POLITICA", users
)

In [50]:

show_from_df(agree_comments)


Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 


  -  @usuario Pena de muerte deberían darle a ese enfermo mental comunista


In [51]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario China te manda el virus
Te cobra el test...no anda
El partido comuns=ista china si sibrevife esto...nos gobernara a todos...
     - SI: ['dvalado', 'gclerici', 'gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

  -  @usuario Pero que se puede esperar de comunistas y de asiaticos
     - SI: ['dvalado', 'gclerici', 'gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Desagradecido 
Le dieron la libertad de casarse con un hombre
Y critica y va en contra del gobierno que le dio ese derecho

El macrismo no te mata
Pero te deja idiota 

de por vida
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favore

In [52]:
%%capture
show_from_df(non_hateful_comments)

## ASPECTO

In [53]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "ASPECTO", users
)

In [54]:
show_from_df(agree_comments)

In [55]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Problema solucionado.
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Un arma tatuada en la cara... Uno menos como dicen muchos...
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Podrían experimentar con estos seres posibles mutaciones y vacunas así hacen su aporte para la sociedad
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['asilva', 'gdamill']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Que asco esos negros 

In [56]:
%%capture
show_from_df(non_hateful_comments)

## CRIMINAL

In [57]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "CRIMINAL", users
)

In [58]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Uno menos.

  -  @usuario Al fin una buena!!!

  -  @usuario Uno menos..

  -  @usuario 👏🏻👏🏻👏🏻 sigan sigan

  -  @usuario Igual no servía para nada......

  -  @usuario Uno menos 💪🏼💪🏼💪🏼

  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Loa beneficios del Covid19

  -  @usuario Deberían de aprovechar lo del coronavirus, y decir que atacó a todos los reclusos y, por eso debieron cremarlos🤷🏻‍♂️

  -  @usuario Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Problema solucionado.

  -  @usuario Una lacra menos 🤷🏼‍♂️

  -  @usuario Un arma tatuada en la cara... Uno menos como dicen muchos...

  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.

  -  @usuario Uno menos 👏🏽👏🏽

  -  @usuario Uno menos

  -  @usuario Si se morían 1000 se solucio

In [59]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Bueno. Sigamos con cosas mas interesantes.
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Qepd ahre
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

  -  @usuario Q enorme perdida
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'gclerici']

  -  @usuario Que lastima
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

  -  @usuario Punto a favor
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: ['dvalado', 'gclerici']

  -  @usuario Pobre ciudadano, seguramente era inoc

In [60]:
%%capture
show_from_df(non_hateful_comments, show=True)

## SALUD


In [61]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "DISCAPACIDAD", users
)

In [62]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria

  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


  -  @usuario Fardin deja la merca,te hace mal. 
le cagaste la vida a un tipo para tener cámara y te cagaste en las víctimas reales.

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 


  -  @usuario Mala imagen sos vos viejo curda, el virus es difícil que llegue a la Argentina dijiste, te tomaste toda la merca que Toma Acsel hijo de un tren lleno de putas


In [63]:
show_from_df(disagree_comments, show=True)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Tal vez después de eso se le pasa la histeria.
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Lleno d Nano Cerebros minusválidos infectados con el peor d todos los Virus e Peroncho KK. Ese cuando se les mete en el cuerpo hace estragos dejándolos inservibles y llenos d fanatismo, lo peor es q cada vez son + y así está el país!! Y no estsoy a favor d MM. Sociedad enferma!!!
     - SI: ['gclerici', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill']

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


  -  @usuario Y tal vez no es culpable, vaya uno a saber,con el circo que hizo esta y lo demente que está la otra que lo denunció primero...
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

Ginés González García admite que prohibieron a los runn

In [64]:
%%capture
show_from_df(non_hateful_comments)

## OTROS

In [65]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "OTROS", users
)

In [66]:
show_from_df(agree_comments)

In [67]:
show_from_df(disagree_comments, show=True)


Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos 


  -  @usuario Abortera
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Negra y abortera
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']


## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [68]:
my_users = users
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for comment_label in CommentLabel.objects.filter(is_hateful=True, article_label__user__username__in=my_users):
    username = comment_label.article_label.user.username
    df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [69]:
kappa_of(df_calls_for_action, users)

(0.6892246362619122, 66)

In [70]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa asilva     - dvalado    = (0.8773594037363275, 181)
Kappa asilva     - fdesanctis = (0.674342105263158, 99)
Kappa asilva     - gclerici   = (0.5854049719326385, 94)
Kappa asilva     - gdamill    = (0.8659859604339504, 105)
Kappa asilva     - lprats     = (0.8313384113166488, 93)
Kappa dvalado    - fdesanctis = (0.6077210460772103, 126)
Kappa dvalado    - gclerici   = (0.4049586776859503, 120)
Kappa dvalado    - gdamill    = (0.6178285009253548, 236)
Kappa dvalado    - lprats     = (0.6789077212806024, 124)
Kappa fdesanctis - gclerici   = (0.5706214689265536, 114)
Kappa fdesanctis - gdamill    = (0.52102919346858, 121)
Kappa fdesanctis - lprats     = (0.656084656084656, 117)
Kappa gclerici   - gdamill    = (0.500943989930774, 122)
Kappa gclerici   - lprats     = (0.5729275970619097, 111)
Kappa gdamill    - lprats     = (0.8466889976810101, 119)


In [71]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
asilva        0.191304
dvalado       0.150611
fdesanctis    0.152672
gclerici      0.074890
gdamill       0.144543
lprats        0.132911
dtype: float64

In [72]:
df = no_null_columns(df_calls_for_action)

df.sum(axis=1)

name
asilva        15.0
dvalado       14.0
fdesanctis    11.0
gclerici       7.0
gdamill       17.0
lprats        15.0
dtype: float64

In [73]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [74]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Que no pare el fuego y rociarlo con nafta es una buena receta..

Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras 


  -  @usuario Bala y al ✨río✨

  -  @usuario Es el momento de tirar una bomba y que exploten todos


In [75]:
show_from_df(no_agree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Parece que este virus nos va a depurar un poco de algunos males...
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Si se morían 1000 se solucionaba el problema de raíz
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Esperemos que sea el primero pero no el último
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']

  -  @usuario Pero si tienen k morir todos los presos
     - SI: ['asilva', '